# Dataset & Model Loading

In [1]:
# #@title Mount your Google Drive
# # If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# # you can delete this cell which is specific to Google Colab. You may also
# # change the paths for data/logs in Arguments below.
# %matplotlib inline
# %load_ext autoreload
# %autoreload 2
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# #@title Link your assignment folder & install requirements
# #@markdown Enter the path to the assignment folder in your Google Drive
# # If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# # you can delete this cell which is specific to Google Colab. 
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/ift6759" #@param {type:"string"}
!ln -Ts "$folder" /content/assignment 2> /dev/null

# # Add the assignment folder to Python path
if '/content/assignment' not in sys.path:
   sys.path.insert(0, '/content/assignment')

# # Check if CUDA is available
import torch
if not torch.cuda.is_available():
   warnings.warn('CUDA is not available.')

In [3]:
!pip3 install torchxrayvision
!pip3 install lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torchxrayvision as xrv # for chest xray pretrained models
import skimage, torch, torchvision
import torchvision.models as models # for general pretrained models

In [5]:
import copy
import os
from skimage.io import imread
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import tqdm
import time
import matplotlib.pyplot as plt
import pandas as pd
import tensorboard
import torchmetrics
from pytorch_lightning.loggers import TensorBoardLogger
from torch import optim, nn, utils, Tensor
from torchvision.transforms import ToTensor
import lightning.pytorch as pl

In [6]:
import warnings
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available')
    use_gpu = False
    device = torch.device('cpu')
else:
    use_gpu = True
    device = torch.device('cuda:0')
torch.cuda.is_available()

True

In [7]:
# What is the task ('classification' or 'regression')
task = 'classification'
# If classification, how many classes?
if task == 'classification':
    final_out_features = 3
# If regression, final out features is 1
elif task == 'regression':
    final_out_features = 1
# What level of dropout do we want for the adaptor (classifier/regressor) head of the model?
head_dropout = 0.5
# Are we using a pretrained model from torchxrayvision?
from_xrv = False
# What is the model name?
#     pretrained models from torchxrayvision:
# ['densenet_all', 'densenet_rsna', 'densenet_nih', 'densenet_pc', 'densenet_chex', 'densenet_mimic_nb', 'densenet_mimic_ch']
#     pretrained models from torchvision:
# ['alexnet','densenet', 'squeezenet', 'mobilenet', 'vgg16']
model_name = 'alexnet'
# How many epochs to train for?
num_epochs = 300
# What optimizer, learning rate and scheduler do we want for training?
learning_rate = 0.001
optimizer_type = 'AdamW' # Choose from 'SGD' or 'AdamW'
# Other train-val-test settings
split_lengths = [0.8, 0.0, 0.2]
batch_size = 10

# Do we want to filter the dataset by chest xray views?
view = ['*'] # [other options are ['PA', 'AP']]

In [8]:
class LightningRegressor(pl.LightningModule):
    def __init__(self,model):
        super().__init__()
        self.model = model
        self.mse = torchmetrics.MeanSquaredError()
        self.mae = torchmetrics.MeanAbsoluteError()
        self.r2_score = torchmetrics.R2Score()
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        # print(f'y: {y}')
        # Make prediction
        x = self.model(x)
        # print(f'x {x}')
        # Calculate and log loss
        loss = nn.functional.mse_loss(x,y)
        mean_squared_error = self.mse(x,y)
        mean_absolute_error = self.mae(x,y)
        r2_score = self.r2_score(x,y)
        to_log = {"train_loss": loss, 
                  "train_mse": mean_squared_error, 
                  "train_mae": mean_absolute_error,
                  'train_r2_score': r2_score}  # add more items if needed
        self.log_dict(to_log)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x,y = batch
        # Make prediction
        x = self.model(x)
        # Calculate and log loss
        loss = nn.functional.mse_loss(x,y)
        mean_squared_error = self.mse(x,y)
        mean_absolute_error = self.mae(x,y)        
        r2_score = self.r2_score(x,y)
        to_log = {"val_loss": loss, 
                  "val_mse": mean_squared_error,
                  "val_mae": mean_absolute_error,
                  'val_r2_score': r2_score}  # add more items if needed
        self.log_dict(to_log)
        return loss
        
    def test_step(self, batch, batch_idx):
        x,y = batch
        # Make prediction
        x = self.model(x)
        # Calculate and log loss
        loss = nn.functional.mse_loss(x,y)
        mean_squared_error = self.mse(x,y)
        mean_absolute_error = self.mae(x,y)        
        r2_score = self.r2_score(x,y)
        to_log = {"test_loss": loss, 
                  "test_mse": mean_squared_error, 
                  "test_mae": mean_absolute_error,
                  'test_r2_score': r2_score}  # add more items if needed
        self.log_dict(to_log)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer
    
class LightningClassifier(pl.LightningModule):
    def __init__(self,model, num_classes):
        super().__init__()
        self.model = model
        self.loss = nn.CrossEntropyLoss()
        self.acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.auroc = torchmetrics.AUROC(task = 'multiclass', num_classes=num_classes)
        self.confusion_matrix = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=3)
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)
        self.precision = torchmetrics.Precision(task="multiclass", num_classes=num_classes)
        self.recall = torchmetrics.Recall(task="multiclass", num_classes=num_classes)
        self.specificity = torchmetrics.Specificity(task="multiclass", num_classes=num_classes)
        self.validation_step_outputs = []
        self.validation_step_targets = []
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        # Make predictions
        x = self.model(x)
        # Calculate and log loss
        loss = self.loss(x,y)
        acc = self.acc(x,y)
        auroc = self.auroc(x,y)
        #confusion_matrix = self.confusion_matrix(x,y)
        f1_score = self.f1_score(x,y)
        precision = self.precision(x,y)
        recall = self.recall(x,y)
        specificity = self.specificity(x,y)
        to_log = {'train_loss': loss,
                  'train_acc': acc,
                  'train_auroc': auroc,
                  #'train_confusion_matrix': confusion_matrix,
                  'train_f1_score': f1_score ,
                  'train_precision': precision,
                  'train_recall': recall,
                  'train_specificity': specificity,
                  }  # add more items if needed
        #print(to_log)
        self.log_dict(to_log)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x,y = batch
        # Make predictions
        x = self.model(x)

        # Calculate and log loss
        loss = self.loss(x,y)
        acc = self.acc(x,y)
        auroc = self.auroc(x,y)
        confusion_matrix = self.confusion_matrix(x,y)
        f1_score = self.f1_score(x,y)
        precision = self.precision(x,y)
        recall = self.recall(x,y)
        specificity = self.specificity(x,y)
        to_log = {'val_loss': loss,
                  'val_acc': acc,
                  'val_auroc': auroc,
                   
                  'val_f1_score': f1_score ,
                  'val_precision': precision,
                  'val_recall': recall,
                  'val_specificity': specificity,
                  }  # add more items if needed
        print(to_log)
        self.log_dict(to_log)

        return loss
        
    def test_step(self, batch, batch_idx):
        x,y = batch
        #print(y)
        # Make predictions
        x = self.model(x)
        #print(x)
        # Calculate and log loss

        loss = self.loss(x,y)
        acc = self.acc(x,y)
        auroc = self.auroc(x,y)
        confusion_matrix = self.confusion_matrix(x,y)
        f1_score = self.f1_score(x,y)
        precision = self.precision(x,y)
        recall = self.recall(x,y)
        specificity = self.specificity(x,y)
        to_log = {
            #'pred': x,
            #       'gt': y,
                  'test_loss': loss,
                  'test_acc': acc,
                  'test_auroc': auroc,
                  #'test_confusion_matrix': str(confusion_matrix),
                  'test_f1_score': f1_score ,
                  'test_precision': precision,
                  'test_recall': recall,
                  'test_specificity': specificity,
                  }  # add more items if needed
        print(to_log)
        self.log_dict(to_log)
        pred = x 
        targets = y 
        self.validation_step_outputs.append(pred)
        self.validation_step_targets.append(targets)
        return pred, targets
    
    #def on_test_epoch_end(self):
        
    #    validation_step_outputs = torch.cat(self.validation_step_outputs)
    #    validation_step_targets = torch.cat(self.validation_step_targets)

        #confusion = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=3)#.to(device='cpu')
     #   cm = self.confusion_matrix(validation_step_outputs, validation_step_targets)
     #   print(cm)
     #   print((validation_step_targets))
     #   print((validation_step_outputs[0].cpu().numpy().argmax()))

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer

In [22]:
class CovidSeverity_Dataset(Dataset):
    def __init__(self, img_dir, annotations_file, from_xrv = False, transform = None, views = ['*'], data_aug = False, task = 'regression'):
        self.image_dir = img_dir
        self.transform = transform
        self.from_xrv = from_xrv
        self.views = views
        self.data_aug = data_aug
        self.task = task

        # Load data labels
        self.csv = pd.read_csv(annotations_file, index_col=0).reset_index()
        
        # Set the labels based on the task
        if self.task == 'regression':
            self.image_labels = self.csv['OpacityScoreGlobal']
        elif self.task == 'classification':
            self.image_labels = self.csv['OpacityScoreGlobal'].round()
            

        # Keep only the selected views.
        self.limit_to_selected_views(views)
        
    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, idx):
        # Read in the image
        image_filename = self.csv['filename'].iloc[idx]
        image_path = os.path.join(self.image_dir, image_filename)

        # If the associated model will be from torchxrayvision, convert to grayscale then normalize
        if False: 
            image = torchvision.io.read_image(image_path, mode = torchvision.io.ImageReadMode.GRAY)
        else:
            image = torchvision.io.read_image(image_path, mode = torchvision.io.ImageReadMode.RGB)
        
        # Convert label to tensor
        label = self.image_labels[idx]
        if self.task == 'regression':
            label = torch.tensor(label, dtype = torch.float).unsqueeze(0)
        elif self.task == 'classification':
            # low
            if float(label) < 2.0:
              label = 0.0
            # moderate 
            if float(label) >= 2.0 and float(label) < 4.0:
              label = 1.0
            # severe
            if float(label) >= 4.0: 
              label = 2.0
            #print(label)
            label = torch.tensor(label).type(torch.LongTensor)   
        
        # Transform image
        if self.transform:
            image = self.transform(image.type(torch.uint8))
        # Apply data augmentation if specified
        if self.data_aug:
            image = self.data_aug(image)
        
        if self.from_xrv:
            image = (image - 127.5) / 127.5 * 1024 # Scales the image to between approx -1024 and 1024, from 0-255
        
        return image.type(torch.float32), label
    
    def limit_to_selected_views(self, views):
        """Filters the images by view based on the values in .csv['view']
        """
        if type(views) is not list:
            views = [views]
        if '*' in views:
            # if you have the wildcard, the rest are irrelevant
            views = ["*"]
        self.views = views

        # missing data is unknown
        self.csv.view.fillna("UNKNOWN", inplace=True)

        if "*" not in views:
            self.csv = self.csv[self.csv["view"].isin(self.views)]  # Select the view

In [23]:
# Define transforms
transform = transforms.Compose([
    transforms.RandomRotation(30),      # rotate +/- 30 degrees
    transforms.RandomHorizontalFlip(),  # rHorizontally flip the given image randomly with a given probability (default p=0.5)
    transforms.RandomAutocontrast(p=0.5),
    transforms.RandomEqualize(p=0.5),
    transforms.CenterCrop(224),
    ])

data_aug = False

# Set up dataset
# If on colab
combined_labels = "/content/gdrive/MyDrive/processed_images/data_processing/final_combined_cxr_metadata.csv"
processed_images = "/content/gdrive/MyDrive/processed_images"

# otherwise
#combined_labels = "./data_processing/combined_cxr_metadata.csv"
#processed_images = "./processed_images"

dataset = CovidSeverity_Dataset(processed_images, 
                                combined_labels, 
                                from_xrv = from_xrv, # defined above, cell 10
                                transform = transform,
                                views = view, # defined above, cell 10
                                data_aug = data_aug, 
                                task = task # defined above, cell 10
                                )

# Split dataset into train, val, test
# Split lengths and batch size are defined above

train_split, val_split, test_split = random_split(dataset= dataset, 
                                                 lengths = split_lengths, # defined above, cell 10
                                                 generator = torch.Generator().manual_seed(42))

train_loader = DataLoader(train_split, batch_size = batch_size, shuffle = True)
#val_loader = DataLoader(val_split, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_split, batch_size = batch_size, shuffle = True)

In [24]:
len(dataset) # 1432

7162

In [25]:
def load_model(model_name):
    """
    Loads a model from torchxrayvision or torchvision
    """
    # Set the chosen model as "model"
    # From torchxrayvision
    ## 224x224 models
    if model_name == 'densenet_all':
        model = xrv.models.DenseNet(weights="densenet121-res224-all")
    elif model_name == 'densenet_rsna':
        model = xrv.models.DenseNet(weights="densenet121-res224-rsna") # RSNA Pneumonia Challenge
    elif model_name == 'densenet_nih':
        model = xrv.models.DenseNet(weights="densenet121-res224-nih") # NIH chest X-ray8
    elif model_name == 'densenet_pc':
        model = xrv.models.DenseNet(weights="densenet121-res224-pc") # PadChest (University of Alicante)
    elif model_name == 'densenet_chex':
        model = xrv.models.DenseNet(weights="densenet121-res224-chex") # CheXpert (Stanford)
    elif model_name == 'densenet_mimic_nb':
        model = xrv.models.DenseNet(weights="densenet121-res224-mimic_nb") # MIMIC-CXR (MIT)
    elif model_name == 'densenet_mimic_ch':
        model = xrv.models.DenseNet(weights="densenet121-res224-mimic_ch") # MIMIC-CXR (MIT)

    # from torchvision
    elif model_name == 'densenet':
        model = models.densenet161(pretrained=True) # torchvision densenet pretrained on imagenet
    elif model_name == 'alexnet':
        model = models.alexnet(pretrained=True) 
    elif model_name == 'squeezenet':
        model = models.squeezenet1_0(pretrained=True)
    elif model_name == 'mobilenet':
        model = models.mobilenet_v2(pretrained=True)
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
    return model

model_name=('densenet')
model = load_model(model_name)

In [26]:
def adapt_model(model, model_name = 'densenet_all', task = 'regression', final_out_features = 1):
    """
    Adapt a model by changing the classification head
    """
    # If the model name is one of the following, it is from torchxrayvision
    if model_name in ['densenet_all', 'densenet_rsna', 'densenet_nih', 'densenet_pc', 
                      'densenet_chex', 'densenet_mimic_nb', 'densenet_mimic_ch']:
        from_xrv = True
        print('here')
    
    # Make a deep copy of the model
    model = copy.deepcopy(model)
    
    # Freeze weights in model
    for param in model.features.parameters():
        param.requires_grad = False
    
    # Get number of features flowing into the classifier layer we want to change
    # If the model is a densenet-161-based model (all of xrv, or densenet from torchvision)
    if model_name in ['densenet_all', 'densenet_rsna', 'densenet_nih', 'densenet_pc', 
                      'densenet_chex', 'densenet_mimic_nb', 'densenet_mimic_ch']:
        # The classifier is named 'classifier'
        in_features = model.classifier.in_features
        hidden_features = int(in_features/2) # note: this constriction was largely arbitrarily-decided
        
        # Kludge on an extra part after self.classifier. xrv gets snippy if you just try to change the classifier itself.
        extra = nn.Sequential(
            nn.Linear(18,hidden_features), # an xrv model outputs 18 features
            nn.ReLU(),
            nn.Linear(hidden_features,final_out_features)
        )
        model = nn.Sequential(model,extra)

    # If the model is one of the following, it can be accessed by .classifier[-1]
    elif model_name in ['alexnet', 'mobilenet', 'vgg16'] :
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, final_out_features)
        
    elif model_name in ['squeezenet'] :
        in_features = model.classifier[-1].in_features
        model.classifier.append(nn.Linear(1000, final_out_features))

    elif model_name == 'densenet':
        in_features = model.classifier.in_features
        hidden_features = int(in_features/2) # note: this constriction was largely arbitrarily-decided
        # Kludge on an extra part after self.classifier, to be comparable with xrv models
        extra = nn.Sequential(
            nn.Linear(1000,hidden_features), # an xrv model outputs 18 features
            nn.ReLU(),
            nn.Linear(hidden_features,final_out_features)
        )
        model = nn.Sequential(model,extra)
    
    return model

In [27]:
model = adapt_model(model, model_name = model_name, task = task, final_out_features = final_out_features)

# Examining the DenseNet out of Box

In [28]:
model
#CUDA_LAUNCH_BLOCKING=1

Sequential(
  (0): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, 

In [29]:
# Initialize the model with pytorch lightning module
num_classes = 3
if task == 'classification':
    lightning_model = LightningClassifier(model, num_classes)
elif task == 'regression':
    lightning_model = LightningRegressor(model)

In [19]:
len(train_loader)

573

In [ ]:
# max train batches can be used 
trainer = pl.Trainer(#limit_train_batches = 100, 
                     max_epochs = 1,
                     accelerator = 'auto',
                     enable_checkpointing = True,
                    #logger = logger,
#                     fast_dev_run = True # runs only one train/val to check code executes ok
                    )
trainer.fit(model = lightning_model, 
            train_dataloaders = train_loader,
            #val_dataloaders = val_loader,
           )



INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name             | Type                      | Params
---------------------------------------------------------------
0 | model            | Sequential                | 29.8 M
1 | loss             | CrossEntropyLoss          | 0     
2 | acc              | MulticlassAccuracy        | 0     
3 | auroc            | MulticlassAUROC           | 0     
4 | conf

Training: 0it [00:00, ?it/s]

# Test Model

In [ ]:
trainer.test(dataloaders=test_loader)
#len(test_loader) 

# VGN16

In [ ]:
model_name=('vgg16')
model = load_model(model_name)
model = adapt_model(model, model_name = model_name, task = task, final_out_features = final_out_features)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth

  0%|          | 0.00/528M [00:00<?, ?B/s]
  2%|▏         | 8.70M/528M [00:00<00:05, 90.9MB/s]
  8%|▊         | 43.4M/528M [00:00<00:02, 251MB/s] 
 15%|█▍        | 78.2M/528M [00:00<00:01, 303MB/s]
 21%|██        | 109M/528M [00:00<00:01, 313MB/s] 
 26%|██▋       | 139M/528M [00:00<00:01, 305MB/s]
 32%|███▏      | 168M/528M [00:00<00:01, 299MB/s]
 37%|███▋      | 197M/528M [00:01<00:04, 71.3MB/s]
 41%|████      | 217M/528M [00:01<00:03, 84.5MB/s]
 45%|████▍     | 237M/528M [00:02<00:06, 46.2MB/s]
 48%|████▊     | 255M/528M [00:02<00:05, 56.9MB/s]
 51%|█████     | 270M/528M [00:03<00:07, 36.1MB/s]
 53%|█████▎    | 281M/528M [00:04<00:10, 25.4MB/s]
 55%|█████▌    | 291M/528M [00:05<00:08, 30.0MB/s]
 57%|█████▋    | 300M/528M [00:07<00:19, 12.5MB/s]
 58%|█████▊    | 306M/528M [00:08<00:22, 10.4MB/s]
 59%|█████▉    | 312M/528M [00:08<00:18, 12.5MB/s]
 60%|██████  

In [ ]:
# Train model
trainer = pl.Trainer(#limit_train_batches = 300, 
                     max_epochs = 1,
                     accelerator = 'auto',
                     enable_checkpointing = True,
#                     fast_dev_run = True # runs only one train/val to check code executes ok
                    )
trainer.fit(model = lightning_model, 
            train_dataloaders = train_loader,
            #val_dataloaders = val_loader,
           )



INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


RuntimeError: ignored

## Standalone Confusion Matrix (with Untrained Model)

In [ ]:
trainer.test(dataloaders=test_loader)

In [ ]:
nb_classes = 3
confusion_matrix = torch.zeros(nb_classes, nb_classes)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

tensor([[   0.,   18.,    0.],
        [   0., 1094.,    0.],
        [   0.,  320.,    0.]])
